In [18]:
# all neccessary imports
import torch
import os
from PIL import Image
import label_converter # make sure the label_converter.py is in the folder with this notebook
import numpy as np
import glob
from torchvision import transforms
import torch.nn as nn

In [2]:
# define paths
PATH_TO_IMAGES = '/Users/aleksandraivanova/Desktop/CMSCB/data'
PATH_TO_MODEL = os.path.join(os.getcwd(), "class_conversion-csv/model.pt")

In [3]:
# load model and print architecture
model = torch.load(PATH_TO_MODEL, map_location=torch.device('cpu'))
model.state_dict

/Users/aleksandraivanova/anaconda3/envs/i2dl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<bound method Module.state_dict of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [4]:
'''# Convert .jpg to .tif
# Input and output folder paths
input_folder = PATH_TO_IMAGES
output_folder = PATH_TO_IMAGES

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        # Create the full path for the input and output files
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".tif")

        # Open the JPG image
        image = Image.open(input_path)

        # Save the image in TIF format
        image.save(output_path, "TIFF")

        # Optionally, you can specify compression options for TIFF
        # image.save(output_path, "TIFF", compression="tiff_deflate")'''

'# Convert .jpg to .tif\n# Input and output folder paths\ninput_folder = PATH_TO_IMAGES\noutput_folder = PATH_TO_IMAGES\n\n# Ensure the output folder exists\nos.makedirs(output_folder, exist_ok=True)\n\n# Loop through all files in the input folder\nfor filename in os.listdir(input_folder):\n    if filename.endswith(".jpg") or filename.endswith(".jpeg"):\n        # Create the full path for the input and output files\n        input_path = os.path.join(input_folder, filename)\n        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".tif")\n\n        # Open the JPG image\n        image = Image.open(input_path)\n\n        # Save the image in TIF format\n        image.save(output_path, "TIFF")\n\n        # Optionally, you can specify compression options for TIFF\n        # image.save(output_path, "TIFF", compression="tiff_deflate")'

In [5]:
converter = label_converter.LabelConverter()

def retrieve_label(filename):
        '''Define dataset-specific function, to retrieve true label for file.
        Here, Extracts data from the conversion table.                  '''
        
        true_lbl = str(filename[:2])
        try:
            result = converter[true_lbl]
            return result
        except KeyError:
            return -1

In [27]:
def create_dataset(root_dirs):
    # Create dataset
    data = []
    for sgl_dir in root_dirs:
        for file_sgl in os.listdir(sgl_dir):

            if not '.tif' in file_sgl:
                        continue

            data.append(os.path.join(sgl_dir, file_sgl))
    
    # Convert the list to a NumPy array
    data = np.array(data)

    # Extract numerical part for sorting
    numeric_part = np.array([int(name.split('image_')[1].split('.tif')[0]) for name in data])

    # Get the indices that would sort the numeric part
    sorted_indices = np.argsort(numeric_part)

    # Use the sorted indices to rearrange the file names array
    sorted_images = data[sorted_indices]

    return sorted_images


In [28]:
def get_image(idx, data):
        '''returns specific item from this dataset'''

        # load image, remove alpha channel, transform
        image = Image.open(data[idx])
        image_arr = np.asarray(image)[:,:,:3]
        image = Image.fromarray(image_arr)

        return torch.tensor(image_arr)

In [31]:
def save_single_cell_probabilities(data, folder_patient):
    array_list = []
    #print("Target \t Prediction")
    for idx in range(len(data)):
        input = get_image(idx, data)
        input = input.permute(2, 0, 1).unsqueeze(0)
        
        # Convert input to float
        input = input.float()
        input = input / 255.

        # Normalize the input
        normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        input = normalize(input)
        
        model.eval()
        pred = model(input)
        softmax = nn.Softmax(dim=1)
        pred_probability = softmax(pred)

        # Save probabilities in a file
        pred_vect = pred_probability.detach().numpy().flatten()
        array_list.append([pred_vect])
        
    #Concatenate all features for one artificial patient    
    single_cell_probs = np.concatenate(array_list,axis=0)
    output_npy_file = folder_patient + '/single_cell_probabilities.npy'
    # Save the array to the .npy file
    np.save(output_npy_file, single_cell_probs)

In [32]:
image_path_list = []
for folder_class in os.listdir(PATH_TO_IMAGES):
    folder_class = os.path.join(PATH_TO_IMAGES, folder_class)
    
    if os.path.isdir(folder_class):
       print(folder_class)
       for folder_patient in os.listdir(folder_class):
            folder_patient = os.path.join(folder_class, folder_patient)
            if os.path.isdir(folder_patient):
                print("Found folder:", folder_patient)
                data = create_dataset([folder_patient])
                save_single_cell_probabilities(data, folder_patient)

/Users/aleksandraivanova/Desktop/CMSCB/data/NPM1
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/ZEQ
(498, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/ZLW
(494, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/XFF
(489, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/EHM
(492, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/ZLQ
(483, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/JTF
(490, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/SYY
(487, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/QBU
(489, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/CVW
(498, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/CZI
(492, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/LBW
(500, 21)
Found folder: /Users/aleksandraivanova/Desktop/CMSCB/data/NPM1/RPA
(498, 21)
Found folder: /Users/aleksa

In [36]:
array_list = []
#print("Target \t Prediction")
for idx in range(len(data)):
    input,label = get_image(idx)
    input = input.permute(2, 0, 1).unsqueeze(0)
    
    # Convert input to float
    input = input.float()
    input = input / 255.

    # Normalize the input
    normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    input = normalize(input)
    
    model.eval()
    pred = model(input)
    softmax = nn.Softmax(dim=1)
    pred_probability = softmax(pred)
    sum = torch.sum(pred_probability)

    # Save probabilities in a file
    pred_vect = pred_probability.detach().numpy().flatten()
    print(pred_vect)
    array_list.append(pred_vect)
    '''class_pred = int(torch.argmax(pred))
    print(f"{label} \t {class_pred}")
    if label == class_pred:
        print(label)'''

Target 	 Prediction


0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
0 	 4
1 	 4
1 	 4
1 	 4


KeyboardInterrupt: 